In [1]:
import numpy as np
import pandas as pd
import subprocess
import requests
from time import sleep

In [2]:
def policy(pod, latency):
    return pod + latency

In [3]:
def generate_intervals(initial, interval_size):
    intervals = []
    intervals.append((0, initial))
    for x in range(interval_size - 1):
        initial_interval_value = intervals[-1][1]
        interval_value = intervals[0][1]
        # To use double value
        # last_interval_value = initial_interval_value * 2 
        last_interval_value = initial_interval_value + interval_value
        intervals.append((initial_interval_value + 1, last_interval_value))

    return intervals

In [4]:
# '''
# build_table(3, 5, 2000, 5)


# pod initial_latency	end_latency	      3     4	    5
#   3               0	       2000    2003  2004	 2005
#   4               0	       2000    2004  2005	 2006
#   5               0	       2000    2005  2006	 2007
#   3            2001	       4000    4003  4004	 4005
#   4            2001	       4000    4004  4005	 4006
#   5            2001	       4000    4005  4006	 4007
#   3            4001	       8000    8003  8004	 8005
#   4            4001	       8000    8004  8005	 8006
#   5            4001	       8000    8005  8006	 8007
#   3            8001	      16000   16003 16004	16005
#   4            8001	      16000   16004 16005	16006
#   5            8001	      16000   16005 16006	16007
#   3           16001	      32000   32003 32004	32005
#   4           16001	      32000   32004 32005	32006
#   5           16001	      32000   32005 32006	32007
# '''
# def build_table(min_pods, max_pods, initial_lat, interval_size):
#     intervals = generate_intervals(initial_lat, interval_size)

#     table = []

#     for y in intervals:
#         for x in range(min_pods, max_pods + 1):
#             options = np.zeros(max_pods + 1 - min_pods)
#             options = [policy(z + x, y[1]) for z in range(len(options))]
            
#             table.append([x, y[0], y[1]] + list(options))

#     labels = ['pod', 'initial_latency', 'end_latency']
#     actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

#     return pd.DataFrame(table, columns=labels+actions)

In [5]:
'''
build_table(3, 5, 2000, 5)


pod initial_latency	end_latency	      3     4	    5
  3               0	       2000    2003  2004	 2005
  4               0	       2000    2004  2005	 2006
  5               0	       2000    2005  2006	 2007
  3            2001	       4000    4003  4004	 4005
  4            2001	       4000    4004  4005	 4006
  5            2001	       4000    4005  4006	 4007
  3            4001	       8000    8003  8004	 8005
  4            4001	       8000    8004  8005	 8006
  5            4001	       8000    8005  8006	 8007
  3            8001	      16000   16003 16004	16005
  4            8001	      16000   16004 16005	16006
  5            8001	      16000   16005 16006	16007
  3           16001	      32000   32003 32004	32005
  4           16001	      32000   32004 32005	32006
  5           16001	      32000   32005 32006	32007
'''
def build_table(min_pods, max_pods, initial_lat, interval_lat_size, initial_req, interval_req_size):
    requests_intervals = generate_intervals(initial_req, interval_req_size)

    table = []

    for request_interval in requests_intervals:
        latency_intervals = generate_intervals(initial_lat, interval_lat_size)
        for latency_interval in latency_intervals:
            for pod in range(min_pods, max_pods + 1):
                options = np.zeros(max_pods + 1 - min_pods)
                options = [0 for opt in range(len(options))]

                table.append([pod, latency_interval[0], latency_interval[1], request_interval[0], request_interval[1]] + list(options))

    labels = ['pod', 'initial_latency', 'end_latency', 'initial_request', 'end_request']
    actions = list(np.arange(min_pods, max_pods + 1).astype(np.str_))

    return pd.DataFrame(table, columns=labels+actions)

In [6]:
'''
find_option(table, 5, 3000)

   3	    4	    5
2003	 2004	 2005
2004	 2005	 2006
2005	 2006	 2007
'''
def find_options(table, pod, latency, request):
    return table[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request)].iloc[:,5::]

In [7]:
'''
find_best_action(table, 5, 3000)

3
'''
def find_best_action(table, pod, latency, request):
    print("find_best_action")
    options = find_options(table, pod, latency, request)
    
    return int(np.max(options).head(1).index.item())

In [8]:
def update_action_result(table, pod, latency, request, action, result):
    print("update_action_result")
    
    table.loc[(table['pod'] == pod) & (table['initial_latency'] <= latency) & (table['end_latency'] >= latency) & (table['initial_request'] <= request) & (table['end_request'] >= request), str(action)] = result
    print("updating pod", pod, " latency ", latency, " request ", request, " result ", result)
    return table

In [9]:
def get_ip():
    # bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return output.decode("utf-8").split('\n')[1:][:-1].pop().split('   ')[3]

In [10]:
'''
return ms latency
'''
def get_latency():
    print("get_latency")
    query = 'rate(http_server_request_duration_seconds_sum{path="/test"}[30s])/rate(http_server_request_duration_seconds_count{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    return float(result) * 1000

In [11]:
get_latency()

get_latency


484.8227008541082

In [12]:
'''
return requests count
'''
def get_request_by_second():
    print("get_request_by_second")    
    query = 'increase(http_server_requests_total{path="/test"}[30s])'
    result = None

    while(True):
        response = requests.get("http://localhost:9090/api/v1/query?query={query}".format(query = query))
        if response.json()['status'] == 'success':
            result = response.json()['data']['result']
            if result == []:
                result = float(0)
            else:
                result = response.json()['data']['result'][0]['value'][1]
                if result == 'NaN': result = float(0)
            break
        sleep(5)
    
    return float(result)

In [13]:
def get_pods():
    print("get_pods")    
    # TODO comando pega quando esta como Terminating
    bashCommand = "kubectl get pods --field-selector=status.phase=Running"
    
    error = True
    output = None
    
    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    
    return len(output.decode("utf-8").split('\n')[1:][:-1])  
    

In [14]:
def set_pods(new_pods):
    print("set_pods")
    bashCommand = "kubectl scale deployment.v1.apps/phpa-web-app-deployment --replicas={pods}".format(pods = new_pods)

    error = True
    output = None

    while(error == True):
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

In [15]:
def run(epochs, table, min_pods, max_pods, reward):
    ideal_latency = 500
    alpha = 0.1
    gamma = 0.6
    epsilon = 0.1
    
    for x in range(epochs):
        latency = get_latency()
        pods = get_pods()
        requests = get_request_by_second()

        print("\n--- Exec ", x)
        print("latency ", latency)
        print("pods ", pods)
        print("requests ", requests)

        if random.uniform(0, 1) < epsilon:
            action = find_best_action(table, pods, latency, requests)
            print("choose action ", action)
        else:
            action = np.random.randint(min_pods, max_pods + 1)
            print("random ", action)

        set_pods(action)
        sleep(100)

        new_latency = get_latency()

        print("action ", action)
        print("new_latency ", new_latency)
        print("requests ", requests)

        reward = get_reward(ideal_latency, pods, action, latency, new_latency)
        print("** reward ", reward, " ***\n")
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        
        table = update_action_result(table, pods, latency, requests, action, reward)

In [16]:
def is_ideal_latency(ideal_latency, latency):
    print("is_ideal_latency")
    return latency <= ideal_latency

In [17]:
def obtained_result(new_value, old_value):
    print("obtained_result")    
    if new_value == old_value:
        return 'kept'
    elif new_value > old_value:
        return 'increased'
    else:
        return 'decreased'

In [18]:
def reward_within_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_within_ideal")
    if not new_latency_is_ideal:
        return -20 # qualquer ação que tomou resultou em levar a latencia para acima do desejado
    
    if pod_state == 'kept': return 10 # não tomou nenhuma ação, mas manteve a latencia ideal
    elif pod_state == 'increased':
        return {
            'kept': -10, # aumentou pods e a latencia se manteve - pode ter aumentado as requisições
            'increased': 0, # aumentou pods e a latencia aumentou - pode ter aumentado as requisições
            'decreased': -20 # ja estava na latencia ideal e aumentou pods desnecessáriamente
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': 20, # diminiu pods e a latencia se manteve
            'increased': 20, # diminiu pods, latencia aumentou mas segue estando dentro do desejado: ótimo cenário
            'decreased': 20 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [19]:
def reward_out_ideal(pod_state, latency_state, new_latency_is_ideal):
    print("reward_out_ideal")    
    if new_latency_is_ideal:
        return 20 # qualquer ação tomada resultou em levar a latencia para dentro do desejado
    
    if pod_state == 'kept':
        return {
            'kept': -20, # não tomou nenhuma ação
            'increased': -20, # não tomou nenhuma ação e aumentou a latencua
            'decreased': -10 # não tomou nenhuma ação mas diminiu a latencia - pode ter diminuido requisições
        }.get(latency_state)
    elif pod_state == 'increased':
        return {
            'kept': 0, # aumentou pods e manteve a latencia - podem ter aumentado as requisições
            'increased': 0, # aumentou pods e aumentou latencia - podem ter aumentado as requisições
            'decreased': 10 # aumentou pods e diminiu a latencia
        }.get(latency_state)
    else: # pod_state == 'decreased'
        return {
            'kept': -10, # diminiu pods e a latencia se manteve
            'increased': -20, # diminiu pods, latencia aumentou
            'decreased': 0 # diminiu pods, diminiu latencia - pode ter diminuido as requisições
        }.get(latency_state)

In [20]:
def get_reward(ideal_latency, old_pods, new_pods, old_latency, new_latency):
    print("get_reward")    
    pod_state = obtained_result(new_pods, old_pods)
    latency_state = obtained_result(new_latency, old_latency)

    old_latency_is_ideal = is_ideal_latency(ideal_latency, old_latency)
    new_latency_is_ideal = is_ideal_latency(ideal_latency, new_latency)

    reward = 0
    if old_latency_is_ideal:
        reward = reward_within_ideal(pod_state, latency_state, new_latency_is_ideal)
    else:
        reward = reward_out_ideal(pod_state, latency_state, new_latency_is_ideal)

    return reward

In [21]:
epochs = 200
min_pods = 1
max_pods = 3
initial_latency = 500
interval_size = 4
initial_request = 18
interval_request = 3
reward = 0

epsilon = 0.5

In [22]:
table = build_table(min_pods, max_pods, initial_latency, interval_size, initial_request, interval_request)
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,18,0,0,0
1,2,0,500,0,18,0,0,0
2,3,0,500,0,18,0,0,0
3,1,501,1000,0,18,0,0,0
4,2,501,1000,0,18,0,0,0
5,3,501,1000,0,18,0,0,0
6,1,1001,1500,0,18,0,0,0
7,2,1001,1500,0,18,0,0,0
8,3,1001,1500,0,18,0,0,0
9,1,1501,2000,0,18,0,0,0


In [27]:
run(epochs, table, min_pods, max_pods, reward)

get_latency
get_pods
get_request_by_second

--- Exec  0
latency  671.375936250115
pods  1
requests  32.0
random  2
set_pods
get_latency
action  2
new_latency  411.0925573364107
requests  32.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  20  ***

update_action_result
updating pod 1  latency  671.375936250115  request  32.0  result  20
get_latency
get_pods
get_request_by_second

--- Exec  1
latency  411.0925573364107
pods  2
requests  18.0
random  3
set_pods
get_latency
action  3
new_latency  456.87658520182595
requests  18.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  0  ***

update_action_result
updating pod 2  latency  411.0925573364107  request  18.0  result  0
get_latency
get_pods
get_request_by_second

--- Exec  2
latency  456.87658520182595
pods  3
requests  10.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  725.3545461668788
r

get_pods
get_request_by_second

--- Exec  20
latency  415.88186099640245
pods  3
requests  12.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  719.4452518306207
requests  12.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  -20  ***

update_action_result
updating pod 3  latency  415.88186099640245  request  12.0  result  -20
get_latency
get_pods
get_request_by_second

--- Exec  21
latency  719.4452518306207
pods  1
requests  24.0
random  2
set_pods
get_latency
action  2
new_latency  469.92003585910425
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  20  ***

update_action_result
updating pod 1  latency  719.4452518306207  request  24.0  result  20
get_latency
get_pods
get_request_by_second

--- Exec  22
latency  469.92003585910425
pods  2
requests  14.0
random  1
set_pods
get_latency
action  1
new_latency  705.0993611652908
re

get_pods
get_request_by_second

--- Exec  40
latency  290.1716555992607
pods  2
requests  10.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  656.5699514603087
requests  10.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  -20  ***

update_action_result
updating pod 2  latency  290.1716555992607  request  10.0  result  -20
get_latency
get_pods
get_request_by_second

--- Exec  41
latency  656.5699514603087
pods  1
requests  30.0
random  2
set_pods
get_latency
action  2
new_latency  311.3067408266943
requests  30.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  20  ***

update_action_result
updating pod 1  latency  656.5699514603087  request  30.0  result  20
get_latency
get_pods
get_request_by_second

--- Exec  42
latency  311.3067408266943
pods  2
requests  11.999999999999998
find_best_action
choose action  1
set_pods
get_latency
action  1

get_pods
get_request_by_second

--- Exec  60
latency  729.6365006598838
pods  1
requests  24.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  718.0401292522826
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -10  ***

update_action_result
updating pod 1  latency  729.6365006598838  request  24.0  result  -10
get_latency
get_pods
get_request_by_second

--- Exec  61
latency  718.0401292522826
pods  1
requests  24.0
random  1
set_pods
get_latency
action  1
new_latency  713.7365352464257
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -10  ***

update_action_result
updating pod 1  latency  718.0401292522826  request  24.0  result  -10
get_latency
get_pods
get_request_by_second

--- Exec  62
latency  713.7365352464257
pods  1
requests  24.0
random  3
set_pods
get_latency
action  3
new_latency  434.5402189937886
request

get_pods
get_request_by_second

--- Exec  80
latency  644.536947503054
pods  1
requests  28.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  723.6708727456668
requests  28.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -20  ***

update_action_result
updating pod 1  latency  644.536947503054  request  28.0  result  -20
get_latency
get_pods
get_request_by_second

--- Exec  81
latency  723.6708727456668
pods  1
requests  24.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  693.3415450039319
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -10  ***

update_action_result
updating pod 1  latency  723.6708727456668  request  24.0  result  -10
get_latency
get_pods
get_request_by_second

--- Exec  82
latency  693.3415450039319
pods  1
requests  24.0
find_best_action
choose action  1
set_pods
get_latency
ac

get_pods
get_request_by_second

--- Exec  100
latency  730.0176515879381
pods  1
requests  24.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  718.5243790057333
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -10  ***

update_action_result
updating pod 1  latency  730.0176515879381  request  24.0  result  -10
get_latency
get_pods
get_request_by_second

--- Exec  101
latency  718.5243790057333
pods  1
requests  24.0
random  2
set_pods
get_latency
action  2
new_latency  470.79147399765344
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  20  ***

update_action_result
updating pod 1  latency  718.5243790057333  request  24.0  result  20
get_latency
get_pods
get_request_by_second

--- Exec  102
latency  470.79147399765344
pods  2
requests  18.0
random  1
set_pods
get_latency
action  1
new_latency  703.0123199995918
requ

get_pods
get_request_by_second

--- Exec  120
latency  726.8236220867644
pods  1
requests  24.0
random  3
set_pods
get_latency
action  3
new_latency  418.08279183169356
requests  24.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  20  ***

update_action_result
updating pod 1  latency  726.8236220867644  request  24.0  result  20
get_latency
get_pods
get_request_by_second

--- Exec  121
latency  418.08279183169356
pods  3
requests  11.999999999999998
random  2
set_pods
get_latency
action  2
new_latency  325.36785800766665
requests  11.999999999999998
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  20  ***

update_action_result
updating pod 3  latency  418.08279183169356  request  11.999999999999998  result  20
get_latency
get_pods
get_request_by_second

--- Exec  122
latency  325.3678580076666
pods  2
requests  12.0
random  1
set_pods
get_latency
action  1
new_latency  7

get_pods
get_request_by_second

--- Exec  140
latency  667.8983228139259
pods  1
requests  28.0
random  1
set_pods
get_latency
action  1
new_latency  667.1165356256097
requests  28.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -10  ***

update_action_result
updating pod 1  latency  667.8983228139259  request  28.0  result  -10
get_latency
get_pods
get_request_by_second

--- Exec  141
latency  667.1165356256097
pods  1
requests  32.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  686.1801851174438
requests  32.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_out_ideal
** reward  -20  ***

update_action_result
updating pod 1  latency  667.1165356256097  request  32.0  result  -20
get_latency
get_pods
get_request_by_second

--- Exec  142
latency  686.1801851174438
pods  1
requests  34.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency

get_pods
get_request_by_second

--- Exec  160
latency  208.1982639792841
pods  3
requests  2.0
random  3
set_pods
get_latency
action  3
new_latency  344.87670400024695
requests  2.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  10  ***

update_action_result
updating pod 3  latency  208.1982639792841  request  2.0  result  10
get_latency
get_pods
get_request_by_second

--- Exec  161
latency  344.87670400024695
pods  3
requests  12.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_latency  764.1235772015838
requests  12.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  -20  ***

update_action_result
updating pod 3  latency  344.87670400024695  request  12.0  result  -20
get_latency
get_pods
get_request_by_second

--- Exec  162
latency  764.1235772015838
pods  1
requests  30.0
find_best_action
choose action  1
set_pods
get_latency
action  1
new_lat

get_pods
get_request_by_second

--- Exec  180
latency  351.6148684939253
pods  3
requests  8.0
random  3
set_pods
get_latency
action  3
new_latency  365.3186830051709
requests  8.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  10  ***

update_action_result
updating pod 3  latency  351.6148684939253  request  8.0  result  10
get_latency
get_pods
get_request_by_second

--- Exec  181
latency  365.3186830051709
pods  3
requests  8.0
random  3
set_pods
get_latency
action  3
new_latency  356.1075272820225
requests  8.0
get_reward
obtained_result
obtained_result
is_ideal_latency
is_ideal_latency
reward_within_ideal
** reward  10  ***

update_action_result
updating pod 3  latency  365.3186830051709  request  8.0  result  10
get_latency
get_pods
get_request_by_second

--- Exec  182
latency  356.1075272820225
pods  3
requests  14.0
random  3
set_pods
get_latency
action  3
new_latency  363.0456292448798
requests  14.0
get_reward
obtain

In [28]:
table

,pod,initial_latency,end_latency,initial_request,end_request,1,2,3
0,1,0,500,0,18,-20,0,0
1,2,0,500,0,18,-20,-20,-20
2,3,0,500,0,18,-20,20,10
3,1,501,1000,0,18,0,0,0
4,2,501,1000,0,18,-20,0,20
5,3,501,1000,0,18,-20,0,20
6,1,1001,1500,0,18,0,0,0
7,2,1001,1500,0,18,0,0,0
8,3,1001,1500,0,18,0,0,0
9,1,1501,2000,0,18,0,0,0


In [24]:
table2 = build_table(min_pods, max_pods, initial_latency, interval_size)
table2

,pod,initial_latency,end_latency,1,2,3
0,1,0,500,501,502,503
1,2,0,500,502,503,504
2,3,0,500,503,504,505
3,1,501,1000,1001,1002,1003
4,2,501,1000,1002,1003,1004
5,3,501,1000,1003,1004,1005
6,1,1001,1500,1501,1502,1503
7,2,1001,1500,1502,1503,1504
8,3,1001,1500,1503,1504,1505
9,1,1501,2000,2001,2002,2003
